In [50]:
#!/usr/bin/env python3
import argparse
import asyncio
import os
import random
import sys
from sys import platform

import keras
import music21
import numpy as np
import pygame
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Dense, Embedding, TimeDistributed, Dropout, Flatten, Input
from keras.models import Sequential, Model
from loguru import logger
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from midi2audio import FluidSynth
from keras.layers import concatenate
from keras.utils import to_categorical


In [51]:
def setup_logging(level="DEBUG", show_module=False):
    """
    Setups better log format for loguru
    """
    logger.remove(0)    # Remove the default logger
    log_level = level
    log_fmt = u"<green>["
    log_fmt += u"{file:10.10}…:{line:<3} | " if show_module else ""
    log_fmt += u"{time:HH:mm:ss.SSS}]</green> <level>{level: <8}</level> | <level>{message}</level>"
    logger.add(sys.stderr, level=log_level, format=log_fmt, colorize=True, backtrace=True, diagnose=True)



def play_midi(midi_file):

    #Play MIDI
    sf2_path = '/usr/share/soundfonts/freepats-general-midi.sf2'  # path to sound font file
    FluidSynth(sound_font=sf2_path).play_midi(midi_file)


In [52]:
sequence_length = 32
batch_size = 32
epochs = 300
# Load the musical data using Music21
corpi = [c for c in random.choices(music21.corpus.getComposer("bach"), k=2)]

X_pitch = []
X_duration = []
y_pitch = []
y_duration = []
notes = []
# for piece in corpus:
for corpus in corpi:
    logger.info(f'Parsing {corpus}')
    parsed = music21.converter.parse(corpus)
    elements = parsed.flat.notes
    for element in tqdm(elements):
        if isinstance(element, music21.note.Note):
            # note = (element.pitch.midi, element.duration.quarterLength)
            # notes.append(note)
            notes.append((element.pitch.midi, element.duration.quarterLength))

# Preprocess the data
X_pitch = []
X_duration = []
y_pitch = []
y_duration = []
for i in tqdm(range(len(notes) - sequence_length)):
    X_pitch.append([pitch for pitch, duration in notes[i:i + sequence_length]])    
    X_duration.append([int(duration / 0.5 + 0.5) for pitch, duration in notes[i:i + sequence_length]])
    y_pitch.append(notes[i + sequence_length][0])
    y_duration.append(int(notes[i + sequence_length][1] / 0.5 + 0.5))

# Convert the data to numpy arrays
X_pitch = np.array(X_pitch)
X_duration = np.array(X_duration)
y_pitch = np.array(y_pitch)
y_duration = np.array(y_duration)

pitch_classes_count = max(np.max(X_pitch), np.max(y_pitch)) + 1
duration_classes_count = max(np.max(X_duration), np.max(y_duration)) + 1

pitch_classes_encoded = to_categorical(y_pitch[:], num_classes=pitch_classes_count)
y_pitch = pitch_classes_encoded

duration_classes_encoded = to_categorical(y_duration[:], num_classes=duration_classes_count)
y_duration = duration_classes_encoded

pitch_classes_encoded = to_categorical(X_pitch[:, :], num_classes=pitch_classes_count)
X_pitch = pitch_classes_encoded

duration_classes_encoded = to_categorical(X_duration[:, :], num_classes=duration_classes_count)
X_duration = duration_classes_encoded

n_samples = (len(X_pitch) // batch_size) * batch_size
X_pitch = X_pitch[:n_samples]
X_duration = X_duration[:n_samples]
y_pitch = y_pitch[:n_samples]
y_duration = y_duration[:n_samples]


2023-02-25 17:25:18.945 | INFO     | __main__:<module>:14 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/bach/bwv60.5.mxl
100%|██████████| 282/282 [00:00<00:00, 197923.98it/s]
2023-02-25 17:25:18.973 | INFO     | __main__:<module>:14 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/bach/bwv393.mxl
100%|██████████| 449/449 [00:00<00:00, 189556.37it/s]


In [53]:
# Define the model architecture
input_pitch = Input(shape=(sequence_length, pitch_classes_count))
input_duration = Input(shape=(sequence_length, duration_classes_count))
merged = concatenate([input_pitch, input_duration])

lstm1 = LSTM(units=128, activation='tanh', return_sequences=True)(merged)
lstm2 = LSTM(units=128)(lstm1)
output_pitch = Dense(pitch_classes_count, activation='softmax', name='pitch')(lstm2)
output_duration = Dense(duration_classes_count, activation='softmax', name='duration')(lstm2)

model = Model(inputs=[input_pitch, input_duration], outputs=[output_pitch, output_duration])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Train the model
history = model.fit([X_pitch, X_duration], [y_pitch, y_duration], batch_size=batch_size, epochs=epochs)


Epoch 1/300
14/14 [==============================] - 2s 19ms/step - loss: 5.3838 - pitch_loss: 4.0233 - duration_loss: 1.3606 - pitch_categorical_accuracy: 0.0647 - duration_categorical_accuracy: 0.5625
Epoch 2/300
14/14 [==============================] - 0s 18ms/step - loss: 4.2732 - pitch_loss: 3.2274 - duration_loss: 1.0459 - pitch_categorical_accuracy: 0.1295 - duration_categorical_accuracy: 0.5536
Epoch 3/300
14/14 [==============================] - 0s 18ms/step - loss: 4.0959 - pitch_loss: 3.0567 - duration_loss: 1.0392 - pitch_categorical_accuracy: 0.1317 - duration_categorical_accuracy: 0.5804
Epoch 4/300
14/14 [==============================] - 0s 18ms/step - loss: 4.0340 - pitch_loss: 3.0077 - duration_loss: 1.0262 - pitch_categorical_accuracy: 0.1317 - duration_categorical_accuracy: 0.5804
Epoch 5/300
14/14 [==============================] - 0s 18ms/step - loss: 4.0143 - pitch_loss: 2.9920 - duration_loss: 1.0224 - pitch_categorical_accuracy: 0.1317 - duration_categorical_ac

In [97]:
from tabnanny import verbose
import numpy as np

def predict_sequence_with_duration(model, seed_sequence, sequence_length, pitch_classes_count, duration_classes_count):
    # Encode the seed sequence
    seed_sequence_pitch = [pitch for pitch, duration in seed_sequence]
    seed_sequence_duration = [duration for pitch, duration in seed_sequence]
    seed_sequence_pitch = to_categorical([seed_sequence_pitch], num_classes=pitch_classes_count)
    seed_sequence_duration = to_categorical([seed_sequence_duration], num_classes=duration_classes_count)

    # Reshape the seed sequence for the model
    # seed_sequence_pitch = seed_sequence_pitch.reshape(1, sequence_length, pitch_classes_count)
    # seed_sequence_duration = seed_sequence_duration.reshape(1, sequence_length, duration_classes_count)

    # Predict the next pitch and duration in the sequence
    predicted_pitch, predicted_duration = model.predict([seed_sequence_pitch, seed_sequence_duration], verbose=0)

    # Decode the predicted pitch and duration
    next_pitch = np.argmax(predicted_pitch[0])
    next_duration = np.argmax(predicted_duration[0])

    return next_pitch, next_duration

def generate_song_with_duration(model, seed_sequence, sequence_length, pitch_classes_count, duration_classes_count, song_length):
    song = seed_sequence
    for i in range(song_length - sequence_length):
        next_pitch, next_duration = predict_sequence_with_duration(model, song[-sequence_length:], sequence_length, pitch_classes_count, duration_classes_count)
        print(next_pitch, next_duration)
        song.append((next_pitch, next_duration))
    return song

# Set the seed sequence and the desired length of the generated song
# seed_sequence = [(60, 1), (62, 1), (64, 1), (65, 1), (67, 0.5), (69, 0.5), (71, 0.5), (72, 0.5)] * 4

seed_sequence = []
for i in range(sequence_length):
    pitch = notes[i][0]
    duration = int(notes[i][1] / 0.5 + 0.5)
    seed_sequence.append((pitch, duration))
song_length = 128

# Generate the song
song = generate_song_with_duration(model, seed_sequence, sequence_length, pitch_classes_count, duration_classes_count, song_length)


1/1 [==============================] - 0s 18ms/step
54 1
1/1 [==============================] - 0s 15ms/step
71 2
1/1 [==============================] - 0s 15ms/step
68 1
1/1 [==============================] - 0s 15ms/step
64 2
1/1 [==============================] - 0s 15ms/step
52 1
1/1 [==============================] - 0s 14ms/step
66 1
1/1 [==============================] - 0s 15ms/step
50 1
1/1 [==============================] - 0s 15ms/step
74 2
1/1 [==============================] - 0s 15ms/step
68 1
1/1 [==============================] - 0s 15ms/step
64 2
1/1 [==============================] - 0s 15ms/step
49 1
1/1 [==============================] - 0s 14ms/step
64 1
1/1 [==============================] - 0s 16ms/step
47 1
1/1 [==============================] - 0s 16ms/step
73 6
1/1 [==============================] - 0s 16ms/step
69 6
1/1 [==============================] - 0s 15ms/step
64 6
1/1 [==============================] - 0s 15ms/step
45 6
1/1 [==========================

In [98]:
# Write the generated stream to a MIDI file

stream = music21.stream.Stream()
for _note in song:
    _note_object = music21.note.Note(_note[0], duration=music21.duration.Duration(_note[1]/2))
    stream.append(_note_object)

stream.write('midi', fp='generated_music.mid')

play_midi('generated_music.mid')

ALSA lib pcm_dsnoop.c:566:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:999:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm_

FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

